In [1]:
import json

def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)
    
data = load_json("prontoqa/json/1hop_ProofsOnly_random_noadj.json")

In [2]:
from random import randint

labels = ["is", "is not"]

def create_adversary(prompt, example):
    question = example['question']
    cot = example['chain_of_thought']
    adv_cot = cot.copy()
    if " are not " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" are not ", " are ")
    elif " is not " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" is not ", " is ")
    elif " are " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" are ", " are not ")
    elif " is " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" is ", " is not ")
    else:
        raise "is not/is haven't been found in string!"

    if cot[1] in question:
        adv_question = question.replace(cot[1], adv_cot[1])
    else:
        raise "CoT not found in question!"
        
    return adv_question + '\n'

def prepare_example(example, n_shots=5, with_cot=True, reverse_cot=False):
    prompt = "Answer True or False to the following question. Answer as in the examples.\n\n"
    cot_prompt = "Think step-by-step.\n\n" if with_cot else ""
    
    for i in range(n_shots):
        prompt += example[f"in_context_example{i}"]['question'] + "\n"
        query = example[f"in_context_example{i}"]['query']
        label = "False" if "is not" in query else "True"
        
        if "is not" in query:
            name, adj = query.split(" is not ")
        else:
            name, adj = query.split(" is ")
        
        name = name.split("Prove: ")[-1]
        query = f"Is {name.strip()} {adj.strip()[:-1]}?\n"
    
        prompt += "Question: " + query + cot_prompt
        cot = example[f"in_context_example{i}"]['chain_of_thought']

        if reverse_cot:
            cot[0], cot[1] = cot[1], cot[0]

        if with_cot:
            for j, step in enumerate(cot):
                prompt += f"({j+1}) {step}\n"
        
        prompt += f"Answer: {label}\n\n"

    adversary = prompt + create_adversary(prompt, example['test_example'])
    prompt += example["test_example"]['question'] + "\n"
    query = example["test_example"]['query']
    label = "False" if "is not" in query else "True"
    cot = example["test_example"]['chain_of_thought']
    if reverse_cot:
        cot[0], cot[1] = cot[1], cot[0]
    
    if "is not" in query:
        name, adj = query.split(" is not ")
    else:
        name, adj = query.split(" is ")

    name = name.split("Prove: ")[-1]
    query = f"Is {name.strip()} {adj.strip()[:-1]}?\n"
    
    prompt += "Question: " + query + cot_prompt
    adversary += "Question: " + query + cot_prompt
        
    return prompt, label, cot, adversary

In [188]:
idx = 0
prompt, label, cot, adv = prepare_example(list(data.values())[idx], n_shots=0, with_cot=False)

In [190]:
print(prompt)

Answer True or False to the following question. Answer as in the examples.

Gorpuses are sterpuses. Gorpuses are grimpuses. Every dumpus is a jompus. Every grimpus is a shumpus. Gorpuses are not small. Sterpuses are liquid. Every shumpus is not muffled. Dumpuses are bright. Each grimpus is a brimpus. Every grimpus is not cold. Wren is a dumpus. Wren is a grimpus.
Question: Is Wren cold?



In [161]:
context = prompt.split('\n')[2]
question = prompt.split('Question:')[-1].strip()
subject = question.split()[1]

In [172]:
relations = [i.strip() for i in context.split('.')[:-1]]

step1 = [relations[-1], relations[-2]]
species = []

for i, step in enumerate(step1):
    species.append((i, step.split()[-1]))

In [173]:
all_species = ['grimpus', 'lorpus', 'wumpus', 'zumpus', 'sterpus', 'numpus', 'jompus', 'brimpus', 'yumpus', 'tumpus', 'dumpus', 'vumpus', 'rompus', 'lempus', 'gorpus', 'shumpus', 'impus']

def is_a_species(x):
    for s in all_species:
        if s in x:
            return True
    return False

step2 = []
existent_species = []
k = 0

while len(species) > 0:
    step2.append([])
    new_species = []
    for rel in relations[:-2]:
        for j, s in species:
            if s not in existent_species:
                existent_species.append(s)

            if s in ' '.join(rel.split()[:-1]).lower():
                step2[k].append((j, rel))
                s_new = rel.split()[-1]
                for s_ in all_species:
                    if s_ in s_new and s_ not in existent_species and s_ not in new_species:
                        new_species.append((len(step2[k])-1, s_))
            
    species = new_species.copy()
    k += 1

In [174]:
step1, step2

(['Polly is an impus', 'Polly is a lorpus'],
 [[(0, 'Impuses are not orange'),
   (1, 'Each lorpus is a dumpus'),
   (1, 'Lorpuses are not luminous'),
   (0, 'Every impus is a jompus'),
   (1, 'Lorpuses are numpuses')],
  [(4, 'Numpuses are not bitter'),
   (4, 'Every numpus is a yumpus'),
   (1, 'Dumpuses are not hot'),
   (4, 'Numpuses are zumpuses')],
  [(1, 'Each yumpus is bright')]])

In [191]:
inference = [i + '.' for i in step1]
final_inference = []

for step in step2:
    new_inference = []
    
    for j, p2p in step:        
        new_inference.append(inference[j] + ' ' + p2p + '.')

    print(new_inference)
    for i, infer in enumerate(new_inference):
        attr = infer[:-1].split()[-1]
        #if not is_a_species(attr):
        final_inference.append(infer)

    inference = new_inference.copy()

['Polly is an impus. Impuses are not orange.', 'Polly is a lorpus. Each lorpus is a dumpus.', 'Polly is a lorpus. Lorpuses are not luminous.', 'Polly is an impus. Every impus is a jompus.', 'Polly is a lorpus. Lorpuses are numpuses.']
['Polly is a lorpus. Lorpuses are numpuses. Numpuses are not bitter.', 'Polly is a lorpus. Lorpuses are numpuses. Every numpus is a yumpus.', 'Polly is a lorpus. Each lorpus is a dumpus. Dumpuses are not hot.', 'Polly is a lorpus. Lorpuses are numpuses. Numpuses are zumpuses.']
['Polly is a lorpus. Lorpuses are numpuses. Every numpus is a yumpus. Each yumpus is bright.']


In [110]:
context

'Gorpuses are sterpuses. Gorpuses are grimpuses. Every dumpus is a jompus. Every grimpus is a shumpus. Gorpuses are not small. Sterpuses are liquid. Every shumpus is not muffled. Dumpuses are bright. Each grimpus is a brimpus. Every grimpus is not cold. Wren is a dumpus. Wren is a grimpus.'

In [ ]:
if 'is' in p2p:
            attr = p2p.split('is')[-1]
        elif 'are' in p2p:
            attr = p2p.split('are')[-1]
        else:
            raise BaseException

 subject + ' is' + attr

```
Answer the question as in the example.

Impuses are not orange. Each rompus is a tumpus. Each lorpus is a dumpus. Each tumpus is large. Every tumpus is a lorpus. Lorpuses are not luminous. Numpuses are not bitter. Every impus is a jompus. Lorpuses are numpuses. Every numpus is a yumpus. Lempuses are melodic. Vumpuses are not angry. Each tumpus is a vumpus. Each yumpus is bright. Rompuses are lempuses. Dumpuses are not hot. Numpuses are zumpuses. Every rompus is not fast. Polly is a lorpus. Polly is an impus.

Question: Is Polly luminous?

Step 1 - Find subject associations
1. Polly is an lorpus.
2. Polly is an impus.

Step 2 - Find species associations
1. Impuses are not orange.
2. Lorpuses are not luminous.
3. Lorpuses are numpuses. Numpuses are not bitter.
4. Each lorpus is a dumpus. Dumpuses are not hot.
5. Lorpuses are numpuses. Every numpus is a yumpus. Each yumpus is bright.

Step 3 - Answer
No, Polly is not luminous.


Gorpuses are sterpuses. Gorpuses are grimpuses. Every dumpus is a jompus. Every grimpus is a shumpus. Gorpuses are not small. Sterpuses are liquid. Every shumpus is not muffled. Dumpuses are bright. Each grimpus is a brimpus. Every grimpus is not cold. Wren is a dumpus. Wren is a grimpus.

Question: Is Wren cold?
```

In [187]:
question

'Is Polly luminous?'

In [ ]:
import json

def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)
    
data = load_json("prontoqa/json/1hop_ProofsOnly_random_noadj.json")

In [ ]:
from random import randint

labels = ["is", "is not"]

def create_adversary(prompt, example):
    question = example['question']
    cot = example['chain_of_thought']
    adv_cot = cot.copy()
    if " are not " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" are not ", " are ")
    elif " is not " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" is not ", " is ")
    elif " are " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" are ", " are not ")
    elif " is " in cot[1]:
        adv_cot[1] = adv_cot[1].replace(" is ", " is not ")
    else:
        raise "is not/is haven't been found in string!"

    if cot[1] in question:
        adv_question = question.replace(cot[1], adv_cot[1])
    else:
        raise "CoT not found in question!"
        
    return adv_question + '\n'

def prepare_example(example, n_shots=5, with_cot=True, reverse_cot=False):
    prompt = "Answer True or False to the following question. Answer as in the examples.\n\n"
    cot_prompt = "Think step-by-step.\n\n" if with_cot else ""
    
    for i in range(n_shots):
        prompt += example[f"in_context_example{i}"]['question'] + "\n"
        query = example[f"in_context_example{i}"]['query']
        label = "False" if "is not" in query else "True"
        
        if "is not" in query:
            name, adj = query.split(" is not ")
        else:
            name, adj = query.split(" is ")
        
        name = name.split("Prove: ")[-1]
        query = f"Is {name.strip()} {adj.strip()[:-1]}?\n"
    
        prompt += "Question: " + query + cot_prompt
        cot = example[f"in_context_example{i}"]['chain_of_thought']

        if reverse_cot:
            cot[0], cot[1] = cot[1], cot[0]

        if with_cot:
            for j, step in enumerate(cot):
                prompt += f"({j+1}) {step}\n"
        
        prompt += f"Answer: {label}\n\n"

    adversary = prompt + create_adversary(prompt, example['test_example'])
    prompt += example["test_example"]['question'] + "\n"
    query = example["test_example"]['query']
    label = "False" if "is not" in query else "True"
    cot = example["test_example"]['chain_of_thought']
    if reverse_cot:
        cot[0], cot[1] = cot[1], cot[0]
    
    if "is not" in query:
        name, adj = query.split(" is not ")
    else:
        name, adj = query.split(" is ")

    name = name.split("Prove: ")[-1]
    query = f"Is {name.strip()} {adj.strip()[:-1]}?\n"
    
    prompt += "Question: " + query + cot_prompt
    adversary += "Question: " + query + cot_prompt
        
    return prompt, label, cot, adversary

In [ ]:
prepare_example(list(data.values())[0], n_shots=3, with_cot=False)

('Answer True or False to the following question. Answer as in the examples.\n\nDumpuses are numpuses. Impuses are jompuses. Each yumpus is not spicy. Every dumpus is mean. Lorpuses are snowy. Each lempus is not transparent. Numpuses are tumpuses. Numpuses are moderate. Every tumpus is luminous. Jompuses are not blue. Impuses are gorpuses. Every gorpus is not hot. Each dumpus is a yumpus. Every gorpus is a lempus. Lorpuses are sterpuses. Every impus is muffled. Every numpus is an impus. Gorpuses are rompuses. Polly is an impus. Polly is a lorpus.\nQuestion: Is Polly muffled?\nAnswer: True\n\nEvery lempus is a rompus. Each rompus is a jompus. Each jompus is a lorpus. Each rompus is a tumpus. Grimpuses are feisty. Jompuses are cold. Each dumpus is transparent. Each lempus is a dumpus. Rompuses are rainy. Vumpuses are gorpuses. Each tumpus is earthy. Every vumpus is sweet. Jompuses are grimpuses. Lempuses are angry. Alex is a rompus. Alex is a vumpus.\nQuestion: Is Alex rainy?\nAnswer: Tr